In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize, root
from scipy.interpolate import interp1d
from cmath import sqrt
from scipy.special import gammaln, logsumexp
import warnings
#from jax import jit, vmap, config, random, device_put, devices
#from jax.lax import fori_loop, cond
#import jax.numpy as jnp
#from jax.experimental.sparse import todense

In [381]:
import numpy as np
import networkx as nx

class SocialNetworkModel:
    def __init__(self, samples,  N, G, Ady, α, β, r_o, maxit):
        
        N = N
        G = G
        r_o = r_o
        β = β
        α = α
        maxit = maxit
        
        Ady = Ady  # Adjacency matrix to represent links
        J = np.zeros_like(Ady)  # Adjacency matrix to represent links
        opinions = np.random.choice([-1, 1], size=(samples, N, G))  # Random binary opinions for each individual
        r_opinions = np.zeros((100, N, G))  # Random binary relevant opinions for each individual
        for i in range(N):
            #print(i)
            self.r_opinions[i][np.random.choice(G, r_o, replace=False)] = 1
    def update_J(self):
            self.J = np.sign(((self.opinions*self.r_opinions) @ (self.opinions*self.r_opinions).T)*self.Ady)
    def update_opinions(self):
        a= 0
        for spin in range(self.N):
            for op in range(self.G):
                #print(i)
                spin = np.random.choice(self.N)
                op = np.random.choice(self.G)
                opinions_new = self.opinions.copy()
                opinions_new[spin][op] = -1*opinions_new[spin][op]
                H = self.compute_H(self.opinions)[spin]
                H_flip = self.compute_H(opinions_new)[spin]
                if np.random.random() <  np.exp(-beta*(H_flip - H)):
                    #a+=1
                    self.opinions = opinions_new
                    self.update_J()
        #return a
    def update_opinion(self, spin, op):
        opinions_new = self.opinions.copy()
        opinions_new[spin][op] = -1*opinions_new[spin][op]
        H = self.compute_H(self.opinions)[spin]
        H_flip = self.compute_H(opinions_new)[spin]
        
        #print(H_flip, H, np.exp(-beta*(H_flip - H)))
        
        if np.random.random() <  np.exp(-beta*(H_flip - H)):
            self.opinions = opinions_new
            self.update_J()
        
    def compute_H(self, opinions):
        return (-self.alpha/self.G*np.sum((opinions@ opinions.T)* self.Ady, axis=1)
                *(np.sum((opinions@ opinions.T)*self.Ady, axis=1)>0) 
                + (1-self.alpha)/self.G*np.sum((opinions@ opinions.T)* self.Ady, axis=1)
                *(np.sum((opinions@ opinions.T)*self.Ady, axis=1)>0))
  
    def compute_H_local(self, opinions, spin, op):
        H_loc=0
        H_loc_flip = 0
        opinions_new = self.opinions.copy()
        opinions_new[spin][op] = -1*opinions_new[spin][op]
        for j in self.opinions[self.Ady[spin]]:
            H_loc+= -self.alpha/self.G * np.sum( (opinions[spin]@j) *((opinions[spin]@j)>0))+(1-self.alpha)/self.G * np.sum( (opinions[spin]@j) *((opinions[spin]@j)<0))
            H_loc_flip+= -self.alpha/self.G * np.sum( (opinions_new[spin]@j) *((opinions_new[spin]@j)>0))+(1-self.alpha)/self.G * np.sum( (opinions_new[spin]@j) *((opinions_new[spin]@j)<0))
        return H_loc_flip, H_loc
    
    def run(self):
        a=0
        for i in range(self.maxit):
            H = np.sum(self.compute_H(self.opinions))
            #print(H)
            self.update_opinions()
            H_ = np.sum(self.compute_H(self.opinions))
            #print(H_)
            if i>10 and (H - H_)<1e-20:
                a+=1
            if a>10:
                #print(i)
                return 1
                break
        
        return 0
            
# Example usage
N = 100  # Number of individuals
G = 3   # Number of binary opinions
r_o = 3
alpha = .0  # Alpha parameter
Ady = np.random.random((N, N))
Ady = Ady<=3/(2*1000)
for i in range(N):
    Ady[i][i]=0
beta = 100
maxit = 100
model = SocialNetworkModel(N, G, Ady, alpha, beta, r_o, maxit)


In [ ]:
ρ = {}
t = {}
t_p = {}
M = {}
k = 3
  # Number of individuals
G = 10
for r_o in range(G):
    ρ[r_o+1] = {}
    t[r_o+1] = {}
    t_p[r_o+1] = {}
    M[r_o+1] = {}
for α in np.arange(0, 1.1, 0.1):
    for r_o in range(G):
        ρ[r_o+1][α] = []
        t_p[r_o+1][α] = []
        t[r_o+1][α] = []
        M[r_o+1][α] = []
    a = 0
    for i in range(100):
        N_1 = 300
        N = N_1.real
        #r_o = 1
        #alpha = .0  # Alpha parameter
        G_1 = nx.erdos_renyi_graph(N_1, k/N_1)
        largest_cc = max(nx.connected_components(G_1), key=len)
        S = G_1.subgraph(largest_cc).copy()
        Ady = np.array(nx.adjacency_matrix(S).toarray())
        N_1 = len(S)
        #print(N)
        β = beta = 10
        maxit = 10000
        for r_o in range(G):
            model = SocialNetworkModel(N_1, G, Ady, α, beta, r_o+1, maxit)
            a+= model.run()
            ρ[r_o+1][α].append(np.sum(model.J[model.J>0])/(np.sum(model.J[model.J>0]) - np.sum(model.J[model.J<0])))
            t[r_o+1][α].append((np.sum(model.J[model.J>0]) - np.sum(model.J[model.J<0])))
            t_p[r_o+1][α].append(np.sum(model.J[model.J>0]))
            max_opinion = [1, 1, 1]
            mag_count = 0
            for j in np.unique(model.opinions, axis=0):
                if np.sum(model.opinions==j)>mag_count:
                    print(j)
                    mag_count = np.sum(model.opinions==j)
                    max_opinion = j
            M[r_o+1][α].append(np.sum(model.opinions@max_opinion)/(N_1*G))
            fig, ax = plt.subplots()
            ax.matshow(model.J)
            ax.set(title=f'Interaction matrix J')
            fig.savefig(f'J_{ r_o = }_{ β = }_{ α = }.png', bbox_inches='tight')
            fig, ax = plt.subplots()
            maping = {j:i for i, j in enumerate(S.nodes())}
            H_1 = nx.relabel_nodes(S, maping)
            m = model.opinions@max_opinion
            pos = nx.circular_layout(H_1)
            colors = {-1: 'red', 1: 'blue', 0: 'grey'}
            edge_color = [colors[model.J[i][j]] for i, j in H_1.edges()]
            nx.draw(H_1, pos, with_labels=False, node_size=10, node_color=m, edge_color=edge_color, cmap=plt.cm.RdBu, vmin = -10, vmax=10)
            fig.savefig(f'Network_{ r_o = }_{ β = }_{ α = }_{N=}_{k=}_.png')
            

/var/folders/dk/__wxlb2x1l5clbbns5cgw5t40000gp/T/ipykernel_21531/859448302.py:28: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Ady = np.array(nx.adjacency_matrix(S).toarray())


32
[-1 -1 -1 -1 -1 -1 -1 -1  1 -1]
32
[-1 -1 -1 -1 -1 -1  1 -1 -1  1]
[-1 -1 -1 -1 -1  1 -1 -1 -1  1]
[-1 -1 -1  1 -1 -1 -1 -1 -1  1]
[-1 -1 -1  1 -1  1 -1 -1 -1 -1]
[-1 -1  1  1 -1  1 -1 -1 -1  1]
35
[-1 -1 -1 -1 -1 -1 -1  1 -1  1]
[-1 -1 -1 -1 -1 -1  1  1 -1 -1]
[-1 -1 -1  1  1 -1  1  1 -1 -1]
[-1  1 -1  1 -1 -1  1  1 -1 -1]
[ 1  1 -1  1 -1  1  1  1 -1 -1]
35
[-1 -1 -1 -1 -1 -1  1 -1 -1  1]
[-1 -1 -1  1 -1 -1  1 -1 -1  1]
[-1 -1  1  1 -1  1 -1 -1 -1  1]
[ 1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[ 1 -1 -1  1 -1  1 -1 -1  1  1]
30
[-1 -1 -1 -1 -1 -1 -1 -1 -1  1]
[-1 -1 -1 -1 -1  1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1  1 -1 -1 -1  1]
[-1  1 -1 -1 -1  1  1  1 -1  1]
[-1  1 -1 -1  1  1 -1 -1 -1  1]
[-1  1 -1 -1  1  1  1 -1 -1  1]
[-1  1  1 -1 -1  1 -1 -1 -1  1]
[-1  1  1 -1 -1  1  1 -1 -1  1]
29
[-1 -1 -1 -1 -1 -1 -1  1 -1  1]
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1]
[-1 -1 -1 -1 -1  1  1 -1 -1 -1]
30
[-1 -1 -1 -1 -1 -1 -1 -1  1  1]
[-1 -1 -1 -1 -1 -1 -1  1  1  1]
[-1 -1 -1 -1  1 -1 -1 -1  1  1]
[-1 -1 -1 -1  1  1 

In [ ]:
ρ_m = {}
tρ_m = {}
t_m = {}
ρ_s = {}
M_m = {}
for r_o in range(G):
    ρ_m[r_o] = []
    M_m[r_o] = []
    tρ_m[r_o] = []
    t_m[r_o] = []
    ρ_s[r_o] = []
    for α in np.arange(0, 1.1, 0.1):
        ρ_m[r_o].append(np.mean(ρ[r_o+1][α]))
        M_m[r_o].append(np.mean(M[r_o+1][α]))
        ρ_s[r_o].append(np.std(ρ[r_o+1][α]))
        tρ_m[r_o].append(np.mean(t_p[r_o+1][α]))
        t_m[r_o].append(np.std(t[r_o+1][α]))

In [ ]:
for r_o in range(G):
    plt.plot(np.arange(0, 1.1, 0.1), ρ_m[r_o],"--", label = "r_o= "+str(r_o+1))
    plt.fill_between(np.arange(0, 1.1, 0.1),
                                np.array(ρ_m[r_o]) - np.array(ρ_s[r_o])/10,
                                    np.array(ρ_m[r_o]) + np.array(ρ_s[r_o])/10, alpha=0.5)
#plt.plot(df['t3']['α'], df['t3']['ρ+'], 'o-', label = 'T = 0.2')
plt.title(f'{G =}_T = 0.2')
plt.xlabel("α")
plt.ylabel("ρ+")
#plt.legend()

In [ ]:
for r_o in range(G):
    plt.plot(np.arange(0, 1.1, 0.1), M_m[r_o],"--", label = "r_o= "+str(r_o+1))
    #plt.fill_between(np.arange(0, 1.1, 0.1),
                                #np.array(ρ_m[r_o]) - np.array(ρ_s[r_o])/10,
                                    #np.array(ρ_m[r_o]) + np.array(ρ_s[r_o])/10, alpha=0.5)
#plt.plot(df['t3']['α'], df['t3']['M'], 'o-', label = 'T = 0.2')
plt.title(f'{G =}_T = 0.2')
plt.xlabel("α")
plt.ylabel("M")
#plt.legend()

In [ ]:
for r_o in range(G):
    plt.plot(np.arange(0, 1.1, 0.1), t_m[r_o], label = "r_o= "+str(r_o+1))
#plt.legend()

In [ ]:
for r_o in range(G):
    plt.plot(np.arange(0, 1.1, 0.1), tρ_m[r_o], label = "r_o= "+str(r_o+1))
#plt.legend()

In [26]:
opinions =np.random.choice([-1, 1], size=(2, 3, 3))  # Random binary opinions for each individual

In [27]:
opinions[0]

array([[ 1,  1,  1],
       [ 1,  1,  1],
       [-1,  1,  1]])

In [37]:
Ady = np.random.random((3, 3))
Ady = Ady<=3/(2*3)
for i in range(3):
    Ady[i][i]=0

In [38]:
(Ady @ opinions)

array([[[-1,  1,  1],
        [ 0,  0,  0],
        [ 1,  1,  1]],

       [[-1, -1, -1],
        [ 0,  0,  0],
        [-1,  1,  1]]])

In [40]:
Ady, opinions

(array([[False, False,  True],
        [False, False, False],
        [False,  True, False]]),
 array([[[ 1,  1,  1],
         [ 1,  1,  1],
         [-1,  1,  1]],
 
        [[ 1,  1, -1],
         [-1,  1,  1],
         [-1, -1, -1]]]))

In [42]:
r_opinions = np.zeros((2, 3, 3))  # Random binary relevant opinions for each individual
        

In [43]:
r_opinions

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [56]:
r_opinions[np.array(np.split(np.array([np.random.choice(3, 2, replace=False) for i in range(2*3)]), 2))] = 1

IndexError: index 2 is out of bounds for axis 0 with size 2